In [31]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset 
import seaborn as sns
import numpy as np
from collections import Counter

In [32]:
X, y = make_multilabel_classification(n_samples=10000, n_features=10, n_classes=3, n_labels=2)
X_torch = torch.FloatTensor(X)
y_torch = torch.FloatTensor(y)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_torch, y_torch, test_size = 0.2)

In [34]:
tensor_x = torch.Tensor(X_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train)

multilabel_data = TensorDataset(tensor_x, tensor_y) # create your datset
train_loader = DataLoader(multilabel_data, batch_size=10, num_workers=4)

In [35]:
# class MultilabelDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y
    
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# multilabel_data = MultilabelDataset(X_train, y_train)
# train_loader = DataLoader(dataset = multilabel_data, batch_size=10)

In [36]:
class MultilabelNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MultilabelNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [37]:
input_dim = X_torch.shape[1]
output_dim = y_torch.shape[1]
model = MultilabelNetwork(input_size=input_dim, hidden_size=20, output_size=output_dim)
model.train()

MultilabelNetwork(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=3, bias=True)
  (sigmoid): Sigmoid()
)

In [38]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
losses = []
slope, bias = [], []
number_epochs = 100
for epoch in range(number_epochs):
    for j, data in enumerate(train_loader):
        
        # optimization
        optimizer.zero_grad()

        # forward pass
        y_hat = model(data[0])

        # compute loss
        loss = loss_fn(y_hat, data[1])
        losses.append(loss.item())

        # backprop
        loss.backward()

        # update weights
        optimizer.step()
    
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}, Loss: {loss.data}")

Epoch 0, Loss: 0.5278902053833008
Epoch 10, Loss: 0.5527245402336121
Epoch 20, Loss: 0.5182768106460571
Epoch 30, Loss: 0.5405674576759338
Epoch 40, Loss: 0.5512539148330688
Epoch 50, Loss: 0.557822585105896
Epoch 60, Loss: 0.5421879291534424
Epoch 70, Loss: 0.5542895197868347
Epoch 80, Loss: 0.5411965847015381
Epoch 90, Loss: 0.5411937832832336


In [43]:
X_test_torch = torch.FloatTensor(X_test)
with torch.no_grad():
    model.eval()
    y_test_hat = model(X_test_torch).round()

In [45]:
y_test_str = [str(i) for i in y_test.detach().numpy()]


In [47]:
most_common_cnt = Counter(y_test_str).most_common()[0][1]
print(f"Naive classifier: {most_common_cnt/len(y_test_str) * 100}%")

Naive classifier: 19.8%


In [48]:
test_acc = accuracy_score(y_test, y_test_hat)
print(f"Test accuracy: {test_acc * 100}%")

Test accuracy: 75.4%
